# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
from collections import OrderedDict

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.CenterCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_image_datasets = datasets.ImageFolder(train_dir,transform=train_transforms)
test_image_datasets=datasets.ImageFolder(test_dir, transform=test_transforms)
valid_image_datasets = datasets.ImageFolder(valid_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloaders = torch.utils.data.DataLoader(train_image_datasets, batch_size=64, shuffle=True)
test_dataloaders = torch.utils.data.DataLoader(test_image_datasets, batch_size=32)
valid_dataloaders = torch.utils.data.DataLoader(valid_image_datasets, batch_size=32)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [5]:
print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [6]:
# TODO: Build and train your network
#load pre-trained model
model=torchvision.models.vgg16(pretrained=True)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:14<00:00, 38175002.09it/s]


In [ ]:
model

In [7]:
#transfer learning, fix pretrained parameters
for param in model.parameters():
    param.requires_grad=False

In [8]:
#define the last classification layer
out_num_layer=len(cat_to_name)
classifier=nn.Sequential(OrderedDict([
                         ('fc1', nn.Linear(25088, 1024)),
                         ('relu1', nn.LeakyReLU()),
                         ('dropout1', nn.Dropout(0.2)),
                         ('fc2',nn.Linear(1024,out_num_layer)),
                         ('log_softmax_out',nn.LogSoftmax(dim=1))]))

model.classifier=classifier

In [ ]:
for param in model.classifier.parameters():
    print(param.requires_grad)

In [9]:
#define loss function and optimizer
model.to('cuda')
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.classifier.parameters(),lr=0.05)


In [ ]:
#train the added classifier

epochs=10
train_losses, test_losses, accuracies=[],[],[]
for e in range(epochs):
    train_loss=0
    model.train()
    for images, labels in train_dataloaders:
               
        images, labels=images.to('cuda'), labels.to('cuda')
        log_ps=model.forward(images)
        loss=criterion(log_ps,labels)
        #print('loss:', loss)
        train_loss+=loss.item()
        
        #before = list(model.classifier.parameters())
        
        optimizer.zero_grad()
        loss.backward()
        #grad=list(model.classifier.parameters())[0].grad
        #print('sum_grad:',grad.sum())
        optimizer.step()
        

        #after = list(model.classifier.parameters())
        #for i in range(len(before)):
            #print(torch.equal(before[i].data, after[i].data))

    
    model.eval()
    test_loss=0
    accuracy=0
    with torch.no_grad():
        for images, labels in test_dataloaders:
            images, labels=images.to('cuda'), labels.to('cuda')
            log_ps=model.forward(images)
            loss=criterion(log_ps, labels)
            test_loss+=loss.item()

            ps=torch.exp(log_ps)
            _,top_class=ps.topk(1,dim=1)
            equals=top_class.view(labels.shape)==labels
            accuracy+=torch.mean(equals.type(torch.FloatTensor))
    
    train_losses.append(train_loss/len(train_dataloaders))
    test_losses.append(test_loss/len(test_dataloaders))
    accuracies.append(accuracy/len(test_dataloaders))
    print("trained: {}/{}..".format(e+1,epochs),
          "train loss: {:.3f}..".format(train_loss/len(train_dataloaders)),
          "test loss: {:.3f}..".format(test_loss/len(test_dataloaders)),
          "accuracy: {:.3f}..".format(accuracy/len(test_dataloaders))
    )
        

In [10]:
epochs = 5
steps = 0
running_loss = 0
print_every = 60
for e in range(epochs):
    # Model in training mode, dropout is on
    model.train()
    for inputs, labels in iter(train_dataloaders):
        steps += 1
        optimizer.zero_grad()
        
        #start = time.time()
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output = model.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        #print(f"Time per batch: {(time.time() - start)/3:.3f} seconds")
        
        if steps % print_every == 0:
            # Model in inference mode, dropout is off
            model.eval()
            
            accuracy = 0
            test_loss = 0
            for ii, (inputs, labels) in enumerate(valid_dataloaders):
                
                inputs, labels = inputs.cuda(), labels.cuda()
                output = model.forward(inputs)
                test_loss += criterion(output, labels).data[0]
                
                ## Calculating the accuracy 
                # Model's output is log-softmax, take exponential to get the probabilities
                ps = torch.exp(output).data
                # Class with highest probability is our predicted class, compare with true label
                equality = (labels.data == ps.max(1)[1])
                # Accuracy is number of correct predictions divided by all predictions, just take the mean
                accuracy += equality.type_as(torch.FloatTensor()).mean()
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(test_loss/len(valid_dataloaders)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(valid_dataloaders)))
            
            running_loss = 0
            
            # Make sure dropout is on for training
            model.train()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 1/5..  Training Loss: 782.575..  Validation Loss: 449.357..  Validation Accuracy: 0.212
Epoch: 2/5..  Training Loss: 324.739..  Validation Loss: 387.754..  Validation Accuracy: 0.277
Epoch: 2/5..  Training Loss: 260.175..  Validation Loss: 296.357..  Validation Accuracy: 0.287
Epoch: 3/5..  Training Loss: 223.009..  Validation Loss: 393.370..  Validation Accuracy: 0.278
Epoch: 3/5..  Training Loss: 258.165..  Validation Loss: 469.683..  Validation Accuracy: 0.326
Epoch: 4/5..  Training Loss: 348.085..  Validation Loss: 514.816..  Validation Accuracy: 0.384
Epoch: 5/5..  Training Loss: 435.063..  Validation Loss: 882.664..  Validation Accuracy: 0.366
Epoch: 5/5..  Training Loss: 442.889..  Validation Loss: 879.363..  Validation Accuracy: 0.361


In [11]:
epochs = 15
steps = 0
running_loss = 0
print_every = 60
for e in range(6,epochs):
    # Model in training mode, dropout is on
    model.train()
    for inputs, labels in iter(train_dataloaders):
        steps += 1
        optimizer.zero_grad()
        
        #start = time.time()
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output = model.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        #print(f"Time per batch: {(time.time() - start)/3:.3f} seconds")
        
        if steps % print_every == 0:
            # Model in inference mode, dropout is off
            model.eval()
            
            accuracy = 0
            test_loss = 0
            for ii, (inputs, labels) in enumerate(valid_dataloaders):
                
                inputs, labels = inputs.cuda(), labels.cuda()
                output = model.forward(inputs)
                test_loss += criterion(output, labels).data[0]
                
                ## Calculating the accuracy 
                # Model's output is log-softmax, take exponential to get the probabilities
                ps = torch.exp(output).data
                # Class with highest probability is our predicted class, compare with true label
                equality = (labels.data == ps.max(1)[1])
                # Accuracy is number of correct predictions divided by all predictions, just take the mean
                accuracy += equality.type_as(torch.FloatTensor()).mean()
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(test_loss/len(valid_dataloaders)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(valid_dataloaders)))
            
            running_loss = 0
            
            # Make sure dropout is on for training
            model.train()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 7/15..  Training Loss: 444.338..  Validation Loss: 1052.101..  Validation Accuracy: 0.381
Epoch: 8/15..  Training Loss: 497.901..  Validation Loss: 1173.994..  Validation Accuracy: 0.382
Epoch: 8/15..  Training Loss: 495.644..  Validation Loss: 1276.395..  Validation Accuracy: 0.423
Epoch: 9/15..  Training Loss: 570.850..  Validation Loss: 1307.912..  Validation Accuracy: 0.454
Epoch: 9/15..  Training Loss: 550.523..  Validation Loss: 1377.529..  Validation Accuracy: 0.450
Epoch: 10/15..  Training Loss: 485.315..  Validation Loss: 1309.564..  Validation Accuracy: 0.484
Epoch: 11/15..  Training Loss: 478.552..  Validation Loss: 1435.928..  Validation Accuracy: 0.470
Epoch: 11/15..  Training Loss: 489.495..  Validation Loss: 1732.964..  Validation Accuracy: 0.483
Epoch: 12/15..  Training Loss: 511.603..  Validation Loss: 1648.591..  Validation Accuracy: 0.462
Epoch: 12/15..  Training Loss: 516.869..  Validation Loss: 1997.049..  Validation Accuracy: 0.454
Epoch: 13/15..  Training 

In [12]:
epochs = 35
steps = 0
running_loss = 0
print_every = 60
for e in range(15,epochs):
    # Model in training mode, dropout is on
    model.train()
    for inputs, labels in iter(train_dataloaders):
        steps += 1
        optimizer.zero_grad()
        
        #start = time.time()
        
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output = model.forward(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        #print(f"Time per batch: {(time.time() - start)/3:.3f} seconds")
        
        if steps % print_every == 0:
            # Model in inference mode, dropout is off
            model.eval()
            
            accuracy = 0
            test_loss = 0
            for ii, (inputs, labels) in enumerate(valid_dataloaders):
                
                inputs, labels = inputs.cuda(), labels.cuda()
                output = model.forward(inputs)
                test_loss += criterion(output, labels).data[0]
                
                ## Calculating the accuracy 
                # Model's output is log-softmax, take exponential to get the probabilities
                ps = torch.exp(output).data
                # Class with highest probability is our predicted class, compare with true label
                equality = (labels.data == ps.max(1)[1])
                # Accuracy is number of correct predictions divided by all predictions, just take the mean
                accuracy += equality.type_as(torch.FloatTensor()).mean()
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(test_loss/len(valid_dataloaders)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(valid_dataloaders)))
            
            running_loss = 0
            
            # Make sure dropout is on for training
            model.train()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 16/35..  Training Loss: 533.617..  Validation Loss: 3751.440..  Validation Accuracy: 0.468
Epoch: 17/35..  Training Loss: 558.985..  Validation Loss: 2643.935..  Validation Accuracy: 0.516
Epoch: 17/35..  Training Loss: 541.130..  Validation Loss: 2511.690..  Validation Accuracy: 0.540
Epoch: 18/35..  Training Loss: 611.335..  Validation Loss: 2741.657..  Validation Accuracy: 0.556
Epoch: 18/35..  Training Loss: 532.676..  Validation Loss: 3106.324..  Validation Accuracy: 0.500
Epoch: 19/35..  Training Loss: 580.086..  Validation Loss: 3347.549..  Validation Accuracy: 0.544
Epoch: 20/35..  Training Loss: 619.126..  Validation Loss: 3127.115..  Validation Accuracy: 0.531
Epoch: 20/35..  Training Loss: 506.403..  Validation Loss: 3325.209..  Validation Accuracy: 0.506
Epoch: 21/35..  Training Loss: 544.771..  Validation Loss: 3045.160..  Validation Accuracy: 0.563
Epoch: 21/35..  Training Loss: 528.818..  Validation Loss: 2551.308..  Validation Accuracy: 0.564
Epoch: 22/35..  Trai

KeyboardInterrupt: 

In [ ]:

# TODO: Build and train your network

'''
Decided to use the VGG-11 pretrained model as this had
fewer layers (11) compared to other models and should 
therefore have quicker training times
'''
model = torchvision.models.vgg11(pretrained=True)
model

In [ ]:
# Weights of pretrained model are frozen so we don't backprop through/update them
for param in model.parameters():
    param.requires_grad = False
    
'''
Changing the number of inputs of the first layer to match the 
number of input features of the pretrained model and changing 
the output of the classifier to match the number of classes in 
this data set:
'''

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 500)),
                          ('relu', nn.ReLU()),
                          ('dropout1', nn.Dropout(0.05)),
                          ('fc2', nn.Linear(500, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

# Replacing the pretrained classifier with the one above
model.classifier = classifier

In [ ]:

# TODO: Train a model with a pre-trained network

# Recommended to use NLLLoss when using Softmax
criterion = nn.NLLLoss()
# Using Adam optimiser which makes use of momentum to avoid local minima
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0008)

In [ ]:

# Implement a function for the validation pass

def validation(model, validloader, criterion):
    valid_loss = 0
    accuracy = 0
    
    # change model to work with cuda
    model.to('cuda')

    # Iterate over data from validloader
    for ii, (images, labels) in enumerate(validloader):
    
        # Change images and labels to work with cuda
        images, labels = images.to('cuda'), labels.to('cuda')

        # Forward pass image though model for prediction
        output = model.forward(images)
        # Calculate loss
        valid_loss += criterion(output, labels).item()
        # Calculate probability
        ps = torch.exp(output)
        
        # Calculate accuracy
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return valid_loss, accuracy

In [ ]:
import time

In [ ]:

epochs = 20
steps = 0
print_every = 40

# change to gpu mode
model.to('cuda')

for e in range(epochs):
    since = time.time()
    running_loss = 0
    
    # Iterating over data to carry out training step
    for ii, (inputs, labels) in enumerate(train_dataloaders):
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        # zeroing parameter gradients
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Carrying out validation step
        if steps % print_every == 0:
            # setting model to evaluation mode during validation
            model.eval()
            
            # Gradients are turned off as no longer in training
            with torch.no_grad():
                valid_loss, accuracy = validation(model, valid_dataloaders, criterion)
                
            print(f"No. epochs: {e+1}, \
            Training Loss: {round(running_loss/print_every,3)} \
            Valid Loss: {round(valid_loss/len(valid_dataloaders),3)} \
            Valid Accuracy: {round(float(accuracy/len(valid_dataloaders)),3)}")
            
            running_loss = 0
            
            # Turning training back on
            model.train()
            
    time_taken = time.time() - since
    print(f"Time taken for epoch: {time_taken} seconds")

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
torch.save({'epoch':e,
            'model_state_dict':model.state_dict(),
            'optimize_state':optimizer.state_dict(),
            'train_loss': train_losses}, 'classifier.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes